# Hand Pose Recognition for Music Control

In [1]:
import utils as u 

2024-04-24 18:35:26.941946: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
file_paths = ["../Data/data1.json", 
              "../Data/data2.json", 
              "../Data/data3.json", 
              "../Data/data4.json", 
              "../Data/data5.json", 
              "../Data/data6.json", 
              "../Data/data7.json",
              "../Data/data8.json", 
              ]

x_data, y_data = u.load_data(file_paths) 
print(y_data) 

[1 1 1 ... 1 1 1]


In [7]:
x_data, y_data = u.shuffle_data(x_data, y_data, random_seed=20) 
print(y_data)

IndexError: index 324681 is out of bounds for axis 0 with size 314857

In [ ]:
split_idx = int(len(x_data) * 0.8)
x_train, y_train, x_test, y_test = u.split_data(x_data, y_data, split_idx)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = u.preprocess_data(x_train, 
                                                                 y_train, 
                                                                 x_test, 
                                                                 y_test)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
print(len(x_train.shape))
print(x_train.max())
print(x_train.min())

print(len(x_test.shape))
print(x_test.max())
print(x_test.min())

In [ ]:
u.explore_data(x_train, y_train, y_test, y_val)

# I Build Baseline

In [ ]:
baseline = u.build_baseline()


In [ ]:
class_weights = {0: 0.5, 
                 1: 1} 

In [ ]:
history, baseline = u.train_model(baseline, x_train, y_train, x_val, y_val, 
                      epochs=10, batch_size=32, class_weights=class_weights) 

In [ ]:
u.plot_loss(history)

# Baseline Evaluation

In [ ]:
test_acc, y_pred = u.test_model(baseline, x_test, y_test)
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict labels for test data
y_pred = u.classify_handpose(baseline, x_test)

# Compute classification report
target_names = ["Target Handpose", "Not Target Handpose"]
report = classification_report(list(y_test), 
                               list(y_pred), 
                               target_names=target_names)

print(report)


In [ ]:
# Classify handpose

# sample random handpose
idx = np.random.randint(0, len(y_test), size=1)[0] 
handpose_features = x_test[idx][np.newaxis, :] 
print('This handpose is not the target handpose.' if y_test[idx]==0 else 
      'This handpose is the target handpose.')

# Use your model to make predictions
y_pred = u.classify_handpose(baseline, handpose_features)

if y_pred[0] == 1:
    print('The model says this handpose is the target handpose.')
else:
    print('The model says this handpose is not the target handpose.')

In [ ]:
print(test_acc)

In [ ]:
baseline.summary()

# II Build a Better Model

In [ ]:
model = u.build_model()

In [ ]:
class_weights = {0: 0.4, # o
                 1: 1} # n

In [ ]:
history, model = u.train_model(model, x_train, y_train, x_val, y_val, 
                      epochs=10, batch_size=32, class_weights=class_weights)

In [ ]:
u.plot_loss(history)

# Model Evaluation

In [ ]:
test_acc, y_pred = u.test_model(model, x_test, y_test)
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict labels for test data
y_pred = u.classify_handpose(model, x_test)

# Compute classification report
target_names = ["Target Handpose", "Not Target Handpose"]
report = classification_report(list(y_test), 
                               list(y_pred), 
                               target_names=target_names)

print(report)

In [ ]:
# Classify handpose

# sample random handpose
idx = np.random.randint(0, len(y_test), size=1)[0] 
handpose_features = x_test[idx][np.newaxis, :] 
print('This handpose is not the target handpose.' if y_test[idx]==0 else 
      'This handpose is the target handpose.')

# Use your model to make predictions
y_pred = u.classify_handpose(baseline, handpose_features)

if y_pred[0] == 1:
    print('The model says this handpose is the target handpose.')
else:
    print('The model says this handpose is not the target handpose.')

In [ ]:
print(test_acc)

In [ ]:
model.summary()

In [ ]:
model.save('./Handpose-Recognition.h5')

In [ ]:
!tensorflowjs_converter \
    --input_format=keras \
    --output_format=tfjs_graph_model \
    --saved_model_tags=serve \
    /Users/qianruzhang/Documents/GitHub/Machine-Learning-ML5/Week5-Real-Time-Handpose-Recognition/ML/Handpose-Recognition.keras \
    /Users/qianruzhang/Documents/GitHub/Machine-Learning-ML5/Week5-Real-Time-Posture-Recognition/tfjsmodel
